In [1]:
import context

import numpy as np
from evalpack.utils.torczon.torczon import solve
from evalpack.metrics import nab
from evalpack.metrics import mti
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class AutoProfile():
    import numpy as np
    from evalpack.utils.torczon.torczon import solve
    def __init__(self, x0, xmin, xmax, Nguess, Niter, initial_box_width=0.1, rbo_p: list = [0.2, 1]) -> None:
        self.rbo_p = rbo_p
        self.x0 = x0
        self.xmin = xmin
        self.xmax = xmax
        self.Nguess = Nguess
        self.Niter= Niter
        self.initial_box_width = initial_box_width

    def compute_approx_rbo(self, ranking, scores, p):
        deltas = np.zeros(len(scores))
        for i_algo in range(len(scores)):
            deltas[i_algo] = sum([1 for j_algo in range(len(scores)) if (i_algo != j_algo) and (scores[i_algo] >= scores[j_algo])])
        vals_gt = np.unique(ranking)
        return sum([(p**i_val)*sum((ranking >= val_rank)*(deltas >= val_rank))/sum(ranking >= val_rank) for i_val, val_rank in enumerate(vals_gt)])

    def compute_RBO_cost(self, ranking, scores):
        return - sum([self.compute_approx_rbo(ranking, scores, iter_p)/self.compute_approx_rbo(ranking, ranking, iter_p) for iter_p in self.rbo_p])
    

    def objective_function(self, var, params):
        scores = np.average(self.raw_scores, weights=var, axis=1)
        return self.compute_RBO_cost(self.ranking, scores)

    def fit(self, raw_ranking, raw_scores):
        ranking = []
        for pos in raw_ranking:
            tmp_ranking = np.zeros(len(pos)) - 1
            ranking = np.concatenate([ranking, tmp_ranking]) + len(pos)
        self.ranking = ranking
        self.raw_scores = raw_scores

        self.solution = solve(self.objective_function, par=None, x0=self.x0, xmin=self.xmin, xmax=self.xmax, Nguess=self.Nguess, Niter=self.Niter, initial_box_width=self.initial_box_width)


In [3]:
raw_ranking = [['A'], ['B'], ['C'], ['D']]
raw_scores = [[0.9, 0.9, 0.7, 0.7], [0.9, 0.6, 0.8, 1], [0, 1, 0.5, 1], [0, 0, 0.3, 0.2]]

auto_mti = AutoProfile(x0=[1,1,1,1], xmin=[0,0,0,0], xmax=[1,1,1,1], Nguess=20, Niter=50, rbo_p=[0.2, 1])

auto_mti.fit(raw_ranking=raw_ranking, raw_scores=raw_scores)

In [4]:
def metric_wrapper(y_true, y_pred, threshold, true_ranking, metric='MTI', rbo_p=[0.2,1], Nguess=20, Niter=50):
    if metric == 'MTI':
        scores = np.zeros((y_pred.shape[1], 4))
        metric = mti.MTI()
        for i_algo in range(y_pred.shape[1]):
            pred = np.zeros(y_pred.shape[0])
            pred[y_pred[:, i_algo] > threshold] = 1
            metric.compute_metrics(labels=y_true, predictions=pred, pos_label=1)
            scores[i_algo, :] = [metric.recall_score, metric.masked_specificity_score, metric.alarm_cardinality_score, metric.anticipation_score,]
        autop = AutoProfile(x0=[1,1,1,1], xmin=[0,0,0,0], xmax=[1,1,1,1], Nguess=Nguess, Niter=Niter, rbo_p=rbo_p)
        autop.fit(raw_ranking=true_ranking, raw_scores=scores)
        return scores, autop.solution.x, autop.solution.f
    elif metric == 'NAB':
        scores = np.zeros((y_pred.shape[1], 3))
        for i_algo in range(y_pred.shape[1]):
            pred = np.zeros(y_pred.shape[0])
            pred[y_pred[:, i_algo] > threshold] = 1
            scores[i_algo, :] = nab.nab_score(y_true=y_true, y_pred=pred, return_sum=False)
        autop = AutoProfile(x0=[1,1,1], xmin=[0,0,0], xmax=[1,1,1], Nguess=Nguess, Niter=Niter, rbo_p=rbo_p)
        autop.fit(raw_ranking=true_ranking, raw_scores=scores)
        return scores, autop.solution.x, autop.solution.f

In [5]:
df_simple = pd.read_csv('../datasets/synthetic_simple_cases.csv')
labels = df_simple.copy().labels.to_numpy(dtype=int)
labels[labels == 1] = 0
labels[labels == -1] = 1
true_ranking = [['A'], ['B'], ['C'], ['D'], ['E']]
scores, weights, cost = metric_wrapper(y_true=labels, y_pred=df_simple.to_numpy()[:, 1:], threshold=0.5, true_ranking=true_ranking, 
               metric='MTI', rbo_p=[0.2, 1], Nguess=20, Niter=50)
print('MTI', weights, cost, np.average(scores, weights=weights, axis=1))
scores, weights, cost = metric_wrapper(y_true=labels, y_pred=df_simple.to_numpy()[:, 1:], threshold=0.5, true_ranking=true_ranking, 
               metric='NAB', rbo_p=[0.2, 1], Nguess=20, Niter=50)
print('NAB', weights, cost, np.average(scores, weights=weights, axis=1))

MTI [0.60295072 0.45543117 0.47402513 0.01122133] -2.0 [0.63252648 0.62227654 0.49355086 0.48468408 0.37420065]
NAB [1. 1. 1.] -1.6181818181818182 [-104.33779523    0.32881694    0.32557811   -1.750345   -177.58277899]


In [6]:
df2 = pd.read_csv('../datasets/synthetic_two_asymetric_areas.csv')
labels = df2.copy().labels.to_numpy(dtype=int)
labels[labels == 1] = 0
labels[labels == -1] = 1
order = ['perfect_score_with_anticipation', 'perfect_score', 'first_half_score', 'shift_100_perfect_score', 'perfect_score_with_FA', 
         'second_half_score', 'first_area_score', 'second_area_score', '1/5_first_area_score', 'random_score']
true_ranking = [['perfect_score_with_anticipation'], ['perfect_score'], ['first_half_score'], ['shift_100_perfect_score'], 
                ['perfect_score_with_FA'], ['second_half_score'], ['first_area_score', 'second_area_score'], ['1/5_first_area_score'], 
                ['random_score']]
scores, weights, cost = metric_wrapper(y_true=labels, y_pred=df2[order].to_numpy(), threshold=0.5, true_ranking=true_ranking, 
               metric='MTI', rbo_p=[0.2, 1], Nguess=20, Niter=50)
print('MTI', weights, cost, np.average(scores, weights=weights, axis=1))
print('MTI with init weights:', np.average(scores, weights=[0.990317,0.92242083,0.8992485,0.1044869 ], axis=1))
scores, weights, cost = metric_wrapper(y_true=labels, y_pred=df2[order].to_numpy(), threshold=0.5, true_ranking=true_ranking, 
               metric='NAB', rbo_p=[0.2, 1], Nguess=20, Niter=50)
print('NAB', weights, cost, np.average(scores, weights=weights, axis=1))
print('NAB with init weights:', np.average(scores, weights=[1,1,1], axis=1))

MTI [1.  1.  0.9 1. ] -1.98211110906311 [0.9933237  0.83419184 0.70598671 0.7052717  0.6167532  0.61538462
 0.54537771 0.54522439 0.39316239 0.28176674]
MTI with init weights: [0.99176485 0.97683276 0.80705287 0.90304667 0.68642186 0.79439366
 0.64656686 0.64654544 0.4987433  0.32450721]
NAB [0.91096929 0.86225105 0.00479952] -1.7459520614533812 [ 0.12130621  1.01098458  1.01098458  1.23064805  0.95103053  0.86923702
  0.02054211  0.02054211 -0.28302335 -0.36546223]
NAB with init weights: [-1.09205134e+02  6.57742865e-01  6.57742865e-01  3.14278727e+01
 -6.74574611e+00  5.65522427e-01 -4.46190062e-03 -4.46190062e-03
 -2.01960477e-01 -1.68813014e+02]


In [7]:
df3 = pd.read_csv('../datasets/synthetic_many_small_areas.csv')
labels = df3.copy().labels.to_numpy(dtype=int)
labels[labels == 1] = 0
labels[labels == -1] = 1
order = ['perfect_score_with_anticipation', 'perfect_score', 'perfect_score_first_half_each_areas', 'shift_100', 'perfect_score_with_FA',
         'perfect_score_second_half_each_areas', 'perfect_score_half_areas', 'random_score']
true_ranking = [['perfect_score_with_anticipation'], ['perfect_score'], ['perfect_score_first_half_each_areas'], ['shift_100'], 
                ['perfect_score_with_FA'], ['perfect_score_second_half_each_areas'], ['perfect_score_half_areas'], ['random_score']]
scores, weights, cost = metric_wrapper(y_true=labels, y_pred=df3[order].to_numpy(), threshold=1, true_ranking=true_ranking, 
               metric='MTI', rbo_p=[0.2, 1], Nguess=20, Niter=50)
print('MTI', weights, cost, np.average(scores, weights=weights, axis=1))
print('MTI with init weights:', np.average(scores, weights=[0.990317,0.92242083,0.8992485,0.1044869 ], axis=1))
scores, weights, cost = metric_wrapper(y_true=labels, y_pred=df3[order].to_numpy(), threshold=1, true_ranking=true_ranking, 
               metric='NAB', rbo_p=[0.2, 1], Nguess=20, Niter=50)
print('NAB', weights, cost, np.average(scores, weights=weights, axis=1))
print('NAB with init weights:', np.average(scores, weights=[1,1,1], axis=1))

MTI [0.37887798 0.62789045 0.50878114 0.53438756] -2.0 [0.98959604 0.8353858  0.7429737  0.66451847 0.66352959 0.64690305
 0.57084161 0.30629742]
MTI with init weights: [0.98925698 0.97737673 0.80759684 0.82745438 0.76194005 0.79439366
 0.64682814 0.31627955]
NAB [2.81669642e-01 6.93361216e-01 2.61626058e-04] -1.7391421588705933 [ 3.89921259  3.98915219  3.98915219  3.53285512  3.98293076  3.4298434
 -2.981909   -9.95297019]
NAB with init weights: [-1.07155069e+02  4.60420006e+00  4.60420006e+00 -1.53471715e+02
 -3.12656675e+00  3.95865699e+00 -3.12333043e-02 -4.66666667e+00]
